### Librerías

In [ ]:
import geopandas as gpd
import pandas as pd
from Valor_catastral_construccion import *
from Valor_catastral_terreno import *
from calculo_predial import *
import random
import numpy as np 


### Base de datos

In [ ]:
path_hector = r"G:\Mi unidad\Equipo_Agua\Geo\Data\Prueba\Muestra_modelo_final.xlsx"
# datos = pd.read_csv(path_hector, encoding='utf8')
datos = pd.read_excel(path_hector)
datos.fillna(0,inplace=True)
datos.columns

In [ ]:
# Obtenemos las columnas de altura
cols_altura = np.array([])
cols_nivel = np.array([])
cols_valor_uni = np.array([])


for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
    elif col.lower().strip().startswith('nivel'):
        cols_nivel = np.append(cols_nivel, col)
    elif col.lower().strip().startswith('val'):
        cols_valor_uni = np.append(cols_valor_uni,col)
len(cols_altura), len(cols_nivel), len(cols_valor_uni)

### Llenado de datos faltantes

In [ ]:
datos['Frente_base'] = datos['Frente']/100
datos['Fondo_base'] = datos['Fondo']/100
datos['Area_base'] = datos['Area_AH']/1000
datos['posicion'] = np.random.choice(['Interior','Intermedio','Esquinero','Frentes_no_contiguos','Cabecero','Manzanero'], len(datos))
datos['grado_conservacion'] = np.random.choice(['Bueno','Normal','Regular','Malo','Ruinoso'], len(datos))
datos['Area_inscrita'] = datos['Area_Model'].map(lambda x: float(x)*random.uniform(0.65,1))

### Calculo de factores

In [ ]:
cols_factor_top = np.array([])
cols_factor_nvl = np.array([])

In [ ]:
# Valor catastral del terreno
datos['Factor_posicion']      = datos['Esquinero|Intermedio'].fillna(0).map(lambda x: get_factor_posicion(x))
datos['Terreno_Posicion']     = datos['Factor_posicion'].map(lambda x: 'Intermedio' if x==0.5 else 'otro')
datos['factor_frente']        = pd.Series(map(factor_frente,datos['Medida_lx'],datos['Terreno_Posicion'] ))
datos['factor_fondo']         = pd.Series(map(factor_fondo,datos['Medida_ly'], datos['Fondo_base'],datos['Terreno_Posicion']))
for col in cols_altura:
    nombre_col = 'Factor_Topografia_'+col
    cols_factor_top = np.append(cols_factor_top, nombre_col)
    datos[nombre_col] = pd.Series(map(factor_topografia,datos[col],datos['Medida_ly'],datos['Terreno_Posicion']))

datos['factor_irregularidad'] = pd.Series(map(factor_irregularidad,datos['Area_Model'], datos['Area_inscrita'], datos['Terreno_Posicion']))
datos['factor_area']          = pd.Series(map(factor_area,datos['Area_Model'], datos['Area_base'],datos['Terreno_Posicion']))
datos['factor_restriccion']   = pd.Series(map(factor_restriccion,datos['Area_Model'], datos['Area_Model'],datos['Terreno_Posicion']))

In [9]:
# Valor catastral de la construcción
datos['factor_grado_conservacion'] = pd.Series(map(factor_grado_conservacion,datos['grado_conservacion']))
datos['factor_edad']               = pd.Series(map(get_factor_edad_va,''))
for col in cols_nivel:
    nombre = 'factor_numero_niveles_'+col
    cols_factor_nvl = np.append(cols_factor_nvl, nombre)
    datos[nombre] = pd.Series(map(get_factor_edad_va,datos[col]))

In [10]:
datos.fillna(0,inplace=True)

### Calculo de valores catastrales

In [11]:
datos.columns
cols_valor_uni = ['Val m2_Man','Val m2_Ban']
cols_factor_top = ['Factor_Topografia_AlturaM_HM','Factor_Topografia_AlturaM__1', 'Factor_Topografia_AlturaM__2',
       'Factor_Topografia_AlturaP_HP', 'Factor_Topografia_AlturaP__1','Factor_Topografia_AlturaIS_H', 
       'Factor_Topografia_AlturaIS_1','Factor_Topografia_AlturaC_HC', 'Factor_Topografia_AlturaR_HR',
       'Factor_Topografia_Altura_min']

In [12]:
cols_using = ['Area_Model','factor_frente','factor_fondo', 'factor_irregularidad','factor_area',
              'Factor_posicion','factor_restriccion']
for col in cols_using:
    datos[col] = datos[col].astype(float)

In [15]:
cols_vc_terr = np.array([])
for col_vu in cols_valor_uni:
    i = 0
    name = col_vu.split('_')[-1]
    for col_ft in cols_factor_top:
        name_col = 'Valor_catastral_terreno_'+name+'_'+str(i)
        datos[name_col] = datos['Area_Model'].astype(float).fillna(0)*datos[col_vu].astype(float).fillna(0)*datos['factor_frente'].astype(float).fillna(0)*datos['factor_fondo'].fillna(0)*datos['factor_irregularidad'].fillna(0)*datos['factor_area'].fillna(0)*datos[col_ft].fillna(0)*datos['Factor_posicion'].fillna(0)*datos['factor_restriccion'].fillna(0)
        i += 1
        cols_vc_terr = np.append(cols_vc_terr,name_col)

In [14]:
datos['Val m2_Man'] = datos['Val m2_Man'].str.replace(',','')

### Exportación de datos

In [ ]:
datos.to_excel(r'C:\Users\ruben\Downloads\Modelo_prueba_v1.xlsx')